# Workflow to Inspect GNSS Time-series vis-a-vis OPERA DISP-S1

- Original code authored by: David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun with 
Extensive modifications by Adrian Borsa and Amy Whetter and other NISAR team members 2022

- Updated by **OPERA DISP-S1 CalVal team (Grace Bato, Jinwoo Kim, Simran Sangha)**, November, 2024


<div class="alert alert-warning">
The initial setup (<b>Prep A</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>



In [ ]:
# Parameters for papermill

### Choose a site from the 'sites' dictionary found 2 cells down
site = 'F08882'
work_dir = './'
mintpy_dir = 'mintpy_output'    # location of mintpy files
output_dir = 'results'          # location to store output figures and text files

calval_sites_csv = 'validation_data/DISP-S1_CalVal_sites.csv'
gnss_csv = f'GNSS_record/{site}.csv'
rejected_gnss_csv_file = f'GNSS_record/{site}_rejectedstations.csv'

# specify GNSS source for validation
from mintpy.objects import gnss
gnss_source = 'UNR'
print(f'Searching for all GNSS stations from source: {gnss_source}')
print(f'May use any of the following supported sources: {gnss.GNSS_SOURCES}')
GNSS = gnss.get_gnss_class(gnss_source)
gnss_dir = f'GNSS-{gnss_source}'

# Mask file used for validation
maskFile = 'mask_temporalCoherence.h5' # maskSpatialCoh.h5, temporalCoherence.h5, inputs/combined_msk.h5

#Set GNSS Parameters
gnss_completeness_threshold = 0.8    #ratio of data timespan with valid GNSS epochs
gnss_residual_stdev_threshold = 10.  #max threshold standard deviation of residuals to linear GNSS fit
gnss_thr_eq = 5.7

# step events (earthquake, volcano)
step_events_date = None      # e.g., for Ridgecrest Earthquake (F18903), ['20190704', '20190706']
if step_events_date is not None and step_events_date:
    step_model = {'polynomial': 1, 'stepdate': step_events_date}
else:  # Added missing colon here
    step_model = None

In [ ]:
# Standard library imports
import math
import os
from datetime import datetime as dt
from pathlib import Path
import warnings

# Third-party imports
import imgkit  # pip install imgkit / conda install -c conda-forge wkhtmltopdf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.colors
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import CRS, Transformer
import rasterio
from affine import Affine
from scipy import signal, stats

# Configure matplotlib
plt.rcParams.update({'font.size': 12})

# Local application/library-specific imports
from mintpy.cli import generate_mask, reference_point, view
from mintpy.objects import timeseries
from mintpy.utils import ptime, readfile, utils as ut, utils0 as ut0
from solid_utils.plotting import display_validation, display_validation_table
from solid_utils.sampling import euclidean_distance, load_geo_utm, profile_samples_utm, samp_pair

# Suppress specific warnings
warnings.filterwarnings("ignore")

## Define CalVal Site 

In [ ]:
### Define list of requirements
## Static for OPERA Cal/Val requirements, do not touch!

# Define secular requirements
secular_gnss_rqmt = 5  # mm/yr for 3 years of data over length scales of 0.1-50 km
gnss_dist_rqmt = [0.1, 50.0]  # km
secular_disp_s1_rqmt = 5  # mm/yr
disp_s1_dist_rqmt = [0.1, 50.0]  # km

# Define temporal sampling requirement
disp_s1_sampling = 12 # days
disp_s1_sampling_percentage = 80 # percentage of acquitions at 12 day sampling (disp_s1_sampling) or better
disp_s1_timespan_requirement = 4 # years

# specify number of DISP-S1 pixels to average for comparison with GNSS
pixel_radius = 0

In [ ]:
################# Set Directories ##########################################
print('\nCurrent directory:', os.getcwd())

if 'work_dir' not in locals():
    work_dir = Path.cwd()

work_dir = os.path.abspath(work_dir)    # absolute path       
print("Work directory:", work_dir)
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)  # Change to Workdir   

output_dir = f'{work_dir}/{output_dir}'     # absolute path of output directory
os.makedirs(output_dir, exist_ok=True)
print("   output  dir:", output_dir)

mintpy_dir = f'{work_dir}/{mintpy_dir}'     # absolute path of mintpy directory 
if not os.path.isdir(mintpy_dir):
    raise FileNotFoundError(f"The folder '{mintpy_dir}' does not exist.")
print("   MintPy  dir:", mintpy_dir)

def force_symlink(src, dst):
    try:
        os.symlink(src, dst)
    except FileExistsError:
        os.unlink(dst)
        os.symlink(src, dst)

# setup symlinks of GNSS folders inside of the MintPy subdirectory
force_symlink(os.path.abspath(gnss_csv),
           f'mintpy_output/{gnss_csv.split("/")[-1]}')
force_symlink(os.path.abspath(rejected_gnss_csv_file),
           f'mintpy_output/{rejected_gnss_csv_file.split("/")[-1]}')
force_symlink(os.path.abspath(gnss_dir),
           f'mintpy_output/{gnss_dir}')
gnss_csv = f'{site}.csv'
rejected_gnss_csv_file = f'{site}_rejectedstations.csv'

############################################################################
### List of OPERA DISP-S1 Cal/Val Sites for secular requirements:
if os.path.exists(calval_sites_csv):
    sites_df = pd.read_csv(calval_sites_csv) 
else:
    raise FileNotFoundError(f"The file {calval_sites_csv} does not exist.")  

display(sites_df)

secular_available_sites = sites_df['site'].values

############################################################################
# hardcode plotting parameters
scale=100
unit='cm'
fontsize=8
cm = 1/2.54 # width X height

## Table of Contents:
<a id='secular_TOC'></a>

<hr/>

[**Prep A. Environment Setup**](#secular_prep_a)

[**1. Load DISP-S1 data**](#load_DISP)

[**2. Find Collocated GNSS Stations**](#load_GNSS)

[**3. Get GNSS Position Time Series**](#get_gnss_TS)

[**4. Project GNSS to LOS Velocities**](#gnss_to_LOS)

[**5. Make Velocity Residuals at GNSS Locations**](#gnss_vel_residuals)
- [5.1. Make Double-Differenced Velocity Residuals](#dd_vel)

[**6. GNSS Timeseries Plots**](#gnss_ts_comparison)

<hr/>

<a id='secular_prep_a'></a>
## Prep A. Environment Setup
Setup your environment for processing data

In [ ]:
#Set Global Plot Parameters
vel_file = os.path.join(mintpy_dir, 'velocity.h5')
disp_s1_ts_file = os.path.join(mintpy_dir, 'timeseries.h5')

if os.path.exists(vel_file) and os.path.exists(disp_s1_ts_file):
    print(f'{vel_file} and {disp_s1_ts_file} exist and we can continue this validation.')
else:
    raise FileNotFoundError(f"The {vel_file} and/or {disp_s1_ts_file} do not exist and are required for this validation.")

msk_file = os.path.join(mintpy_dir, maskFile)  # maskTempCoh.h5 maskSpatialCoh.h5 maskConnComp.h5 waterMask.h5

if site not in secular_available_sites:
    msg = '\nSelected site not available! Please select one of the following sites:: \n{}'.format(secular_available_sites)
    raise Exception(msg)
else:
    print('\nSelected site: {}'.format(site))
    display(sites_df[sites_df['site'] == site])

os.chdir(mintpy_dir)  # move to MintPy directory 

### Load plotting functions

In [ ]:
def _rmse(predictions, targets):
    return np.sqrt(np.nanmean((np.ma.masked_invalid(predictions) - np.ma.masked_invalid(targets)) ** 2))


def plot_histogram(ax, difference, bins=20, scale=1e2, unit='cm', fontsize=6):
    # histogram of differences
    ax.hist(difference*scale, bins=bins, color='red', alpha=0.5)
    ax.set_xlabel(f'Diff. [{unit}]', fontsize=fontsize, labelpad=-0.2)
    ax.tick_params(direction='in', labelsize=fontsize, length=2, pad=1.5)
    ax.axvline(difference.mean()*scale, color='darkred', linestyle='--', label='Mean')
    return ax


def plot_scatterplot(ax,
                     data1_ts, data2_ts,
                     label1, label2,
                     scale=1e2, fontsize=6, 
                     unit='cm', **kwargs):
    # Convert to pandas
    data1_df = pd.DataFrame(data1_ts).T
    data1_df = data1_df.rename(columns={0:'date1', 1:'disp1'})

    data2_df = pd.DataFrame(data2_ts).T
    data2_df = data2_df.rename(columns={0:'date2', 1:'disp2'})

    # Merge, and keep common dates
    merged_df = pd.merge(data1_df, data2_df,
                         left_on='date1', right_on='date2',
                         how='inner')

    # Get min, and max
    df_min = np.round(np.min(merged_df[['disp1', 'disp2']].min()), 2)
    df_max = np.round(np.max(merged_df[['disp1', 'disp2']].max()), 2)
    ax_range = np.nanmax(np.abs([df_min, df_max]))
    ax_range += ax_range*0.2 # increase by 20%
    ax_lims = [-ax_range*scale, ax_range*scale]

    # text upper corner
    txt_xy = ax_range- ax_range*0.1
    txt_xy *= scale

    yrange = (ax_range*2) * scale
    r = yrange/20

    # Replace pandas nat with nan
    merged_df['disp1'] = merged_df['disp1'].replace({pd.NaT: np.nan})
    merged_df['disp2'] = merged_df['disp2'].replace({pd.NaT: np.nan})

    # Plot
    ax.plot(merged_df.disp1*scale, merged_df.disp2*scale, 'o', **kwargs)
    
    ax.plot(ax_lims, ax_lims, lw=0.5, color='navy')
    ax.set_xlabel(f'{label1} [{unit}]', fontsize=fontsize, labelpad=-0.1)
    ax.set_ylabel(f'{label2} [{unit}]', fontsize=fontsize, labelpad=-0.1)
    ax.set_xlim(ax_lims)
    ax.set_ylim(ax_lims)

    # Get stats
    merged_df = merged_df.dropna()
    rmse = _rmse(merged_df.disp1.values, merged_df.disp2.values)
    mad = stats.median_abs_deviation(merged_df.disp1 - merged_df.disp2)
    r2 = stats.pearsonr(np.float64(merged_df.disp1), np.float64(merged_df.disp2))[0]


    ax.text(-txt_xy, txt_xy, f'R2: {r2*scale:.2f}', weight='bold', fontsize=fontsize-1)
    ax.text(-txt_xy, txt_xy-r, f'RMSE: {rmse*scale:.2f} {unit}', weight='bold', fontsize=fontsize-1)
    ax.text(-txt_xy, txt_xy-r*2, f'MAD: {mad*scale:.2f} {unit}', weight='bold', fontsize=fontsize-1)
    ax.tick_params(direction='in', labelsize=fontsize, length=2)
    return merged_df


def _plot_subplots(fig, subplot_pos):
    spec = GridSpec(ncols=2, nrows=1, figure=fig)
    ax =  {}
    for p in subplot_pos.keys(): ax[p] = fig.add_subplot(spec[0,0])
    for fig_label in ax: ax[fig_label].set_position(subplot_pos[fig_label])
    return fig, ax


def add_colorbar(fig, loc, im, labelsize=4, label='mm/yr', **kwargs):
    warnings.simplefilter("ignore")
    cb_kwargs = dict(orientation='horizontal', extend='neither')
    cbar_ax = fig.add_axes(loc)
    cb = fig.colorbar(im, cax=cbar_ax, **{**cb_kwargs, **kwargs})
    tick_locator = matplotlib.ticker.MaxNLocator(nbins=3)
    cb.locator = tick_locator
    cb.set_ticklabels(cb.get_ticks(), fontsize=labelsize)
    if label: cb.set_label(label, size=labelsize)
    cb.ax.tick_params(axis="both", grid_color='white',
                       which='major', labelsize=labelsize, length=2)
    cb.outline.set_edgecolor('black')
    cb.outline.set_linewidth(0.5)
    cb.update_ticks()
    return cb

<a id='load_DISP'></a>
## 1. Load DISP-S1 data

In [ ]:
# reading area of DISP-S1
disp_s1_metadata = readfile.read_attribute(vel_file)

assert 'UTM_ZONE' in disp_s1_metadata.keys()  # make sure data in UTM zone

DISP_region = list(ut.four_corners(disp_s1_metadata))

geo_S, geo_W = ut0.utm2latlon(disp_s1_metadata, DISP_region[2], DISP_region[0])
geo_N, geo_E = ut0.utm2latlon(disp_s1_metadata, DISP_region[3], DISP_region[1])
DISP_region_geo = (geo_S, geo_N, geo_W, geo_E)

print(f'region of DISP-S1 (UTM): {DISP_region}, Zone: {disp_s1_metadata["UTM_ZONE"]}')
print('region of DISP-S1 (lat/lon): ', DISP_region_geo)

In [ ]:
# load velocity file
disp_s1_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  # read velocity file
disp_s1_velocities = disp_s1_velocities * 1000.  # convert velocities from m to mm/yr

# set masked pixels to NaN
msk,_ = readfile.read(msk_file)
disp_s1_velocities[msk == 0] = np.nan
disp_s1_velocities[disp_s1_velocities == 0] = np.nan

<a id='load_GNSS'></a>
## 2. Find Collocated GNSS Stations

The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

In [ ]:
# get analysis metadata from DISP-S1 velocity file
disp_s1_metadata = readfile.read_attribute(vel_file)

start_date = disp_s1_metadata.get('START_DATE', None)
end_date = disp_s1_metadata.get('END_DATE', None)
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")

geom_file = os.path.join(mintpy_dir, 'geometryGeo.h5')
inc_angle = readfile.read(geom_file, datasetName='incidenceAngle')[0]
inc_angle = np.nanmean(inc_angle)
az_angle = readfile.read(geom_file, datasetName='azimuthAngle')[0]
az_angle = np.nanmean(az_angle)

if os.path.exists(gnss_csv):
    gnss_df = pd.read_csv(gnss_csv)
    rejected_gnss_df = pd.read_csv(rejected_gnss_csv_file)
    # dummy-proof by discarding rejected stations tracked in the other csv file
    gnss_df = gnss_df[~gnss_df['site'].isin(rejected_gnss_df['site'])]
    gnss_df = gnss_df.reset_index(drop=True)
else:
    raise FileNotFoundError(f"{gnss_csv}- Not Found and should be created by run0_gnss_download_screen.py")

site_names = gnss_df['site']
site_lats_wgs84 = gnss_df['lat']
site_lons_wgs84 = gnss_df['lon']

# post-query: convert lat/lon to UTM for plotting
site_north, site_east = ut0.latlon2utm(disp_s1_metadata, site_lats_wgs84, site_lons_wgs84)

site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

<a id='get_gnss_TS'></a>
## 3. Get GNSS Position Time Series


In [ ]:
# get daily position solutions for GNSS stations
use_stn = []  #stations to keep
bad_stn = []  #stations to toss
use_north = [] 
use_east = []
# track latlon coordinates for UTM grids
use_lats_keepwgs84 = [] 
use_lons_keepwgs84 = []
# get array dim
insar_shape = [int(disp_s1_metadata['LENGTH']),
               int(disp_s1_metadata['WIDTH'])]
insar_coord = ut.coordinate(disp_s1_metadata)

for counter, stn in enumerate(site_names):
    gps_obj = GNSS(site = stn,
                   data_dir = os.path.join(mintpy_dir,f'GNSS-{gnss_source}'))
    gps_obj.open(print_msg=False)

    # get station lat/lon
    gps_lat, gps_lon = gps_obj.get_site_lat_lon()
    gps_y, gps_x = insar_coord.geo2radar(gps_lat, gps_lon)[:2]
    
    # only proceed if station is within valid bounds
    if gps_y >= insar_shape[0] or gps_x >= insar_shape[1]:
        print(f'Skipping {stn} since it is outside of valid TS bounds')
        bad_stn.append(stn)
        continue
    
    # for this quick screening check of data quality, we use the constant incidence and azimuth angles 
    # get standard deviation of residuals to linear fit
    disp_los = ut.enu2los(gps_obj.dis_e, gps_obj.dis_n, gps_obj.dis_u, inc_angle, az_angle)
    disp_detrended = signal.detrend(disp_los)
    stn_stdv = np.std(disp_detrended)

    # to remove NaN gnss velocity
    gnss_velocity = gnss.get_los_obs(disp_s1_metadata, 
                            'velocity', 
                            [stn], 
                            start_date=start_date,
                            end_date=end_date,
                            source=gnss_source,
                            gnss_comp='enu2los', 
                            model = step_model, 
                            redo=False, print_msg=False)
    
    # count number of dates in time range
    dates = gps_obj.dates
    range_days = (end_date_gnss - start_date_gnss).days
    gnss_count = np.histogram(dates, bins=[start_date_gnss, end_date_gnss])
    gnss_count = int(gnss_count[0])

    # select GNSS stations based on data completeness and scatter of residuals
    if range_days * gnss_completeness_threshold <= gnss_count:
        if (stn_stdv > gnss_residual_stdev_threshold) or np.isnan(gnss_velocity):
            bad_stn.append(stn)
        else:
            use_stn.append(stn)
            use_north.append(site_north[counter])
            use_east.append(site_east[counter])
            use_lats_keepwgs84.append(site_lats_wgs84[counter])
            use_lons_keepwgs84.append(site_lons_wgs84[counter])
    else:
        bad_stn.append(stn)

site_names = use_stn
site_north = use_north
site_east = use_east
site_lats_wgs84 = use_lats_keepwgs84
site_lons_wgs84 = use_lons_keepwgs84

print("\nFinal list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_stn)))
print(bad_stn)

<a id='gnss_to_LOS'></a>
## 4. Project GNSS to LOS Velocities

In [ ]:
gnss_velocities = gnss.get_los_obs(disp_s1_metadata, 
                            'velocity', 
                            site_names, 
                            start_date=start_date,
                            end_date=end_date,
                            source=gnss_source,
                            gnss_comp='enu2los', 
                            model = step_model, 
                            redo=True)

# scale site velocities from m/yr to mm/yr
gnss_velocities *= 1000.

print('\n site   vel_los [mm/yr]')
print(np.array([site_names, gnss_velocities]).T)

In [ ]:
# reference GNSS stations to GNSS reference site
ref_site = sites_df[sites_df["site"]==site]["gps_ref_site_name"].values[0]
ref_site_ind = site_names.index(ref_site)
gnss_velocities = gnss_velocities - gnss_velocities[ref_site_ind]

# reference disp_s1 to GNSS reference site
ref_site_lat = float(site_lats_wgs84[ref_site_ind])
ref_site_lon = float(site_lons_wgs84[ref_site_ind])
ref_site_north = float(site_north[ref_site_ind])
ref_site_east = float(site_east[ref_site_ind])

ref_y, ref_x = ut.coordinate(disp_s1_metadata).geo2radar(ref_site_lat, ref_site_lon)[:2]     # x/y location of reference on velocity
if not math.isnan(disp_s1_velocities[ref_y, ref_x]):
    #disp_s1_velocities = disp_s1_velocities - disp_s1_velocities[ref_y, ref_x]
    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    ref_vel_px_rad = disp_s1_velocities[ref_y-pixel_radius:ref_y+1+pixel_radius, 
                        ref_x-pixel_radius:ref_x+1+pixel_radius]
    ref_disp_s1_site_vel = np.nanmedian(ref_vel_px_rad)
    if np.isnan(ref_disp_s1_site_vel):
        ref_disp_s1_site_vel = 0.
    disp_s1_velocities = disp_s1_velocities - ref_disp_s1_site_vel

<a id='gnss_vel_residuals'></a>
## 5. Make Velocity Residuals at GNSS Locations


In [ ]:
#Create dictionary with the stations as the key and all their info as an array 
stn_dict = {}

insar_vel_list = []
gnss_vel_list = []

#Loop over GNSS station locations
for i in range(len(site_names)): 
    # convert GNSS station lat/lon information to velocity x/y grid
    stn_lat = float(site_lats_wgs84[i])
    stn_lon = float(site_lons_wgs84[i])

    y_value, x_value = ut.coordinate(disp_s1_metadata).geo2radar(stn_lat, stn_lon)[:2]
    stn_north = float(site_north[i])
    stn_east = float(site_east[i])
    
    # get velocities and residuals
    gnss_site_vel = gnss_velocities[i]
    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    vel_px_rad = disp_s1_velocities[y_value-pixel_radius:y_value+1+pixel_radius, 
                     x_value-pixel_radius:x_value+1+pixel_radius]
    disp_s1_site_vel = np.nanmedian(vel_px_rad)
    if not np.isnan(disp_s1_site_vel):        # when only displacement exists
        residual = gnss_site_vel - disp_s1_site_vel

        # populate data structure
        values = [x_value, y_value, disp_s1_site_vel, gnss_site_vel, residual, stn_north, stn_east]
        stn = site_names[i]
        stn_dict[stn] = values
        gnss_vel_list.append(gnss_site_vel)
        insar_vel_list.append(disp_s1_site_vel)

# extract data from structure
res_list = []
disp_s1_site_vels = []
gnss_site_vels = []
north_list = []
east_list = []
site_names_used = []    

for stn in stn_dict.keys(): 
    disp_s1_site_vels.append(stn_dict[stn][2])
    gnss_site_vels.append(stn_dict[stn][3])
    res_list.append(stn_dict[stn][4])
    north_list.append(stn_dict[stn][5])
    east_list.append(stn_dict[stn][6])
    site_names_used.append(stn)

num_stn = len(site_names_used) 
site_names_removed = list(set(site_names) - set(site_names_used))

print(f"The GNSS sites ({num_stn} stations) will be used for residual analysis: \n {site_names_used}")
print(f"The GNSS sites  ({len(site_names_removed)} stations) are removed due to the absence of DISP-S1 velocity: \n {site_names_removed}")

print('Finish creating DISP-S1 residuals at GNSS sites')

In [ ]:
# plot scatter plot of InSAR vs GNSS
fig, ax = plt.subplots(1,1, figsize=(6,3), dpi=300)
df1 = plot_scatterplot(ax, 
                    (range(len(gnss_vel_list)), gnss_vel_list), 
                    (range(len(gnss_vel_list)), insar_vel_list),
                    'GNSS vel', 'DISP-S1 vel',
                    ms=1, fontsize=fontsize-2,
                    scale=0.1, unit=f'{unit}/yr')

plt.savefig(f'{output_dir}/GNSS_vs_DISPvel_scatter.jpg',
            bbox_inches='tight', pad_inches=0.1)
plt.show()
plt.close()
del fig
del ax

<a id='dd_vel'></a>
### 5.1. Make Double-Differenced Velocity Residuals


In [ ]:
n_gps_sites = len(site_names_used)
diff_res_list = []
stn_dist_list = []

# loop over stations
for i in range(n_gps_sites-1):
    stn1 = site_names_used[i]
    for j in range(i + 1, n_gps_sites):
        stn2 = site_names_used[j]

        # calculate GNSS and DISP-S1 velocity differences between stations
        gps_vel_diff = stn_dict[stn1][3] - stn_dict[stn2][3]
        disp_s1_vel_diff = stn_dict[stn1][2] - stn_dict[stn2][2]

        # calculate GNSS vs DISP-S1 differences (double differences) between stations
        diff_res = gps_vel_diff - disp_s1_vel_diff
        diff_res_list.append(diff_res)

        # get euclidean distance (km) between stations
        # index 5 is northing, 6 is easting
        stn_dist = euclidean_distance(stn_dict[stn1][6], stn_dict[stn1][5],
                                      stn_dict[stn2][6], stn_dict[stn2][5])
        stn_dist_list.append(stn_dist)

# Write data for statistical tests
gnss_site_dist = np.array(stn_dist_list)
double_diff_rel_measure = np.array(np.abs(diff_res_list))
ndx = np.argsort(gnss_site_dist)

<a id='gnss_ts_comparison'></a>
## 6. GNSS Timeseries Plots


In [ ]:
gnss_ts_plots_flag = True  # if gnss timeseries will be plotted. Reading timeseries may require large memory

if gnss_ts_plots_flag:
    # hardcode positions
    subplots_positions = {'ts': [0.03, 0.22, 0.73, 0.76],
                        'sp': [0.83, 0.25, 0.16, 0.73],
                        'hist': [0.83, 0.01, 0.16, 0.15],
                        'ts_dif': [0.03, 0.02, 0.73, 0.17]}

    # grab the time-series file used for time function estimation given the template setup
    template = readfile.read_template(os.path.join(mintpy_dir, 'smallbaselineApp.cfg'))
    template = ut.check_template_auto_value(template)

    # read the time-series file
    disp_s1_ts, atr = readfile.read(disp_s1_ts_file, datasetName='timeseries')
    mask = readfile.read(os.path.join(mintpy_dir, maskFile))[0]
    print(f'reading timeseries from file: {disp_s1_ts_file}')

    # Get date list
    date_list = timeseries(disp_s1_ts_file).get_date_list()
    num_date = len(date_list)
    date0, date1 = date_list[0], date_list[-1]
    #!#disp_s1_dates = ptime.date_list2vector(date_list)[0]
    disp_s1_dates = pd.to_datetime(date_list, format='%Y%m%d')

    # spatial reference
    coord = ut.coordinate(atr)
    ref_gnss_obj = GNSS(site=ref_site,
                        data_dir=os.path.join(mintpy_dir, f'GNSS-{gnss_source}'))
    ref_lat, ref_lon = ref_gnss_obj.get_site_lat_lon()
    ref_y, ref_x = coord.geo2radar(ref_lat, ref_lon)[:2]
    if not np.any(mask[ref_y-pixel_radius:ref_y+1+pixel_radius, ref_x-pixel_radius:ref_x+1+pixel_radius]):
        raise ValueError(f'Given reference GNSS site ({ref_site}) '
                        'is in masked-out unrelible region in DISP-S1! '
                        'Change to a different site.')

    #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    #To fix, remove the station in section 4 when the site_names list is filtered
    OG_ref_disp_s1_dis = disp_s1_ts[:, ref_y-pixel_radius:ref_y+1+pixel_radius, 
                                ref_x-pixel_radius:ref_x+1+pixel_radius]
    ref_disp_s1_dis = np.zeros(len(OG_ref_disp_s1_dis))
    for i in range(len(OG_ref_disp_s1_dis)):
        ts_med_slice = np.nanmedian(OG_ref_disp_s1_dis[i])
        if np.isnan(ts_med_slice):
            ts_med_slice = 0.
        ref_disp_s1_dis[i] = ts_med_slice

    # Plot displacements and velocity timeseries at GNSS station locations
    num_site = len(site_names_used)
    prog_bar = ptime.progressBar(maxValue=num_site)
    for i, site_name in enumerate(site_names_used):
        prog_bar.update(i+1, suffix=f'{site_names_used} {i+1}/{num_site}')

        ## read data
        # read GNSS
        gnss_obj = GNSS(site=site_name,
                        data_dir=os.path.join(mintpy_dir, f'GNSS-{gnss_source}'))
        gnss_dates, gnss_dis, _, gnss_lalo = gnss_obj.get_los_displacement(
            atr, start_date=date0, end_date=date1, ref_site=ref_site)[:4]
        gnss_dates = pd.to_datetime(gnss_dates, format='%Y%m%d') #!#
        # shift GNSS to zero-mean in time [for plotting purpose]
        gnss_dis -= np.nanmedian(gnss_dis)

        # read DISP-S1
        y, x = coord.geo2radar(gnss_lalo[0], gnss_lalo[1])[:2]
        disp_s1_dis = disp_s1_ts[:, y, x] - ref_disp_s1_dis
        # apply a constant shift in time to fit DISP-S1 to GNSS
        comm_dates = sorted(list(set(gnss_dates) & set(disp_s1_dates)))
        if comm_dates:
            disp_s1_flag = [x in comm_dates for x in disp_s1_dates]
            gnss_flag = [x in comm_dates for x in gnss_dates]
            disp_s1_dis -= np.nanmedian(disp_s1_dis[disp_s1_flag] - gnss_dis[gnss_flag])
            diff_disp_gnss = disp_s1_dis[disp_s1_flag] - gnss_dis[gnss_flag]

        ## plot figure
        if gnss_dis.size > 0 and np.any(~np.isnan(disp_s1_dis)):
            # initiate figure
            fig = plt.figure(figsize=(18*cm, 6*cm), layout="none", dpi=300)
            fig, ax = _plot_subplots(fig, subplots_positions)

            # plot InSAR
            ax['ts'].plot(disp_s1_dates, disp_s1_dis*scale, '.',
                        color='blue', label='DISP-S1', ms=3)
            #
            ax['ts'].set_ylabel(f'Disp. [{unit}]', fontsize=fontsize)
            ax['ts'].set_title(
                f'GNSS {site_name} at lat: {gnss_lalo[0]:.4f}, '
                f'lon: {gnss_lalo[1]:.4f}',
                fontsize=fontsize
            )
            ax['ts'].tick_params(labelsize=fontsize, labelbottom=False)
            ax['ts'].axhline(0, color='gray', lw=0.3, linestyle='--')

            # plot GNSS
            ax['ts'].plot(gnss_dates, gnss_dis*scale, '.',
                        color='orange', ms=3, zorder=0, label='GNSS')
            ax['ts'].legend(fontsize=fontsize)

            # plot scatter plot TS comparison
            df_comp = plot_scatterplot(ax['sp'],
                                ([gnss_dates, gnss_dis]),
                                ([disp_s1_dates, disp_s1_dis]), 
                                'GNSS', 'DISP-S1',
                                ms=1, fontsize=fontsize-2,
                                scale=scale, unit=unit)

            # histogram of differences
            ax['hist'] = plot_histogram(ax['hist'],
                                        diff_disp_gnss,
                                        scale=scale,
                                        unit=unit,
                                        fontsize=fontsize-2)

            # Differences
            ax['ts_dif'].bar(df_comp.date1,
                             diff_disp_gnss*scale,
                             width=np.timedelta64(10, 'D'),
                             color='red',
                             label='DISP-S1 - GNSS')
            ax['ts_dif'].set_ylabel(f'Diff. [{unit}]', fontsize=fontsize)
            ax['ts_dif'].tick_params(labelsize=fontsize)
            ax['ts_dif'].set_xticks(ax['ts'].get_xticks())
            ax['ts_dif'].set_xlim(ax['ts'].get_xlim())
            ax['ts_dif'].axhline(0, color='r', lw=0.5, linestyle='--')
            ax['ts_dif'].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y/%m'))
            ax['ts_dif'].legend(fontsize=fontsize-2)

            # save and plot figure
            plt.savefig(f'{output_dir}/GNSS_{site_name}_vs_DISP.jpg',
                        bbox_inches='tight', pad_inches=0.1)
            plt.show()
            plt.close()
            del fig
            del ax
    prog_bar.close()